Today we are going to visualize my life using Fitbit and Matplotlib. 

# What is Fitbit

[Fitbit](https://www.fitbit.com) is a fitness watch that tracks your sleep, heart rate, and activity.
Fitbit is able to track your steps, however, it is also able to detect multiple types of activity
like running, walking, "sport" and biking.

# What is Matplotlib

[Matplotlib](https://matplotlib.org/) is a python visualization library that enables you to create bar graphs, line graphs, distributions and many more things.
Being able to visualize your results is essential to any person working with data at any scale.
Although I like [GGplot](https://ggplot2.tidyverse.org/) in R more than Matplotlib, Matplotlib is still my go to graphing library for Python. 

# Getting Your Data

There are two main ways that you can get your Fitbit data:

- Fitbit API
- Data Archival Export


Since connecting to the API and setting up all the web hooks can be a pain, I'm just going to use the data export option because this is only for one person.
You can export your data here: [https://www.fitbit.com/settings/data/export](https://www.fitbit.com/settings/data/export).

![Data export on fitbit's website](dataExport.png)

The Fitbit data archive was very organized and kept meticulous records of everything. 
All of the data was organized in separate JSON files labeled by date.
Fitbit keeps around 1MB of data on you per day; most of this data is from the heart rate sensors.
Although 1MB of data may sound like a ton of data, it is probably a lot less if you store it in formats other than JSON. 
When I downloaded the compressed file it was 20MB, but when I extracted it, it was 380MB!
I've only been using Fitbit for 11 months at this point. 

![compressed data](compression.png)

## Sleep

Sleep is something fun to visualize.
No matter how much of it you get you still feel tired as a college student.



## Resting Heart Rate

## Steps